In [ ]:
import torch

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from predictor import SamPredictor
from sam import build_sam
import numpy as np

In [ ]:
sam = build_sam()
_ = sam.eval()
sam.load_state_dict(torch.load('test_sam6.pth', map_location='cpu'))
predictor = SamPredictor(sam)

In [ ]:
test_image_path = "/private/home/mintun/DalleBunny.png"

In [ ]:
from detectron2.data.detection_utils import read_image

In [ ]:
im = read_image(test_image_path, "RGB")

In [ ]:
predictor.set_image(im)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
point_coords = torch.tensor([[[700.0,100.0]]]).numpy()
point_labels = torch.tensor([[1]]).numpy()
#point_coords = torch.tensor([[[500,400], [600, 700]]])
#point_labels = torch.tensor([[2,3]])

In [ ]:
masks, iou_preds, low_res_masks = predictor.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    boxes=None,
    mask_input=None,
    multimask_output=True,
)

In [ ]:
masks.shape

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(im)
plt.scatter(point_coords[0,:,0], point_coords[0,:,1], color='green', s=100)
plt.show()

In [ ]:
for m in masks[0]:
    plt.figure(figsize=(20,20))
    plt.imshow(m)
    plt.scatter(point_coords[0,:,0], point_coords[0,:,1], color='green', s=100)
    plt.show()

In [ ]:
from segment_everything import InteractivePredictor
from detectron2.config import LazyConfig

In [ ]:
cfg = LazyConfig.load("/checkpoint/segment_everything/models/95701_config.yaml")
checkpoint = "/checkpoint/segment_everything/models/95701_model_final.pth"
old_model = InteractivePredictor.from_cfg(cfg, checkpoint, device='cpu')

In [ ]:
old_model.set_image(im)

In [ ]:
torch.all(predictor.features == old_model.features['low_res_embeddings'])

In [ ]:
old_model_masks, old_model_iou_preds, old_model_for_next_iter = old_model.predict(
        point_coords=point_coords[0],
        point_labels=point_labels[0],
        from_prev_iter=None,
        return_best_mask_only=False,
        fix_point_eval_of_box_model=True,
        threshold=True,
        is_first_iter=True,
)

In [ ]:
old_model_masks.shape

In [ ]:
for m in old_model_masks:
    plt.figure(figsize=(20,20))
    plt.imshow(m)
    plt.scatter(point_coords[0,:,0], point_coords[0,:,1], color='green', s=100)
    plt.show()

In [ ]:
low_res_masks.shape

In [ ]:
old_model_low_res_masks = old_model_for_next_iter["low_res_pred_masks"]

In [ ]:
np.all(low_res_masks== old_model_low_res_masks.numpy())

In [ ]:
old_model_low_res_masks.shape

In [ ]:
old_model_masks.shape

In [ ]:
np.any(masks ^ old_model_masks)

In [ ]:
for m in (masks ^ old_model_masks)[0]:
    plt.figure(figsize=(20,20))
    plt.imshow(m)
    plt.scatter(point_coords[0,:,0], point_coords[0,:,1], color='green', s=100)
    plt.show()